# Customer Churn Prediction

## Dataset Intregration using SQL

In [1]:
# imports
import sqlite3

import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import get_scorer

import tabpy

In [2]:
# create new sqlite self-contained database
con = sqlite3.connect("cust_analysis.db")

In [3]:
# read in the datasets into pandas dataframe
customer_df = pd.read_csv("../data/customer.csv")
print(customer_df.shape)

subscription_df = pd.read_csv("../data/subscription.csv")
print(subscription_df.shape)

campaign_df = pd.read_csv("../data/campaign.csv")
print(campaign_df.shape)

service_df = pd.read_csv("../data/service_call.csv")
print(service_df.shape)

(30000, 7)
(120152, 11)
(10529, 14)
(30780, 9)


In [4]:
# storing dataframes in the created sqlite database
customer_df.to_sql(name="customer", con=con, index=False, if_exists="replace")
subscription_df.to_sql(name="subscription", con=con, index=False, if_exists="replace")
campaign_df.to_sql(name="campaign", con=con, index=False, if_exists="replace")
service_df.to_sql(name="service_call", con=con, index=False, if_exists="replace")

30780

In [5]:
# creating an integrated dataset
query = """
--- join all tables
with cs_summary as (
    select 
        month_sqn, 
        customer_sqn, 
        count(sr_number) as num_cs_tickets
    from service_call 
    group by month_sqn, customer_sqn
), 

campaign_summary as (
    select 
        month_sqn, 
        customer_sqn, 
        subscription_sqn, 
        count(distinct campaign_code) as num_campaigns, 
        count(distinct promo_code) as num_promos,
        count(distinct bundle_code) as num_bundles
    from campaign 
    group by month_sqn, customer_sqn, subscription_sqn
)
select 
    s.*,
    c.gender, 
    c.age, 
    c.race, 
    c.nationality, 
    c.credit_rating, 
    cs.num_cs_tickets, 
    camp.num_campaigns, 
    camp.num_promos, 
    camp.num_bundles
from subscription s
left join campaign_summary camp using (customer_sqn, month_sqn, subscription_sqn) 
left join customer c using (customer_sqn, month_sqn)
left join cs_summary cs using (customer_sqn, month_sqn)
"""
integrated_df = pd.read_sql(query, con)

print(integrated_df.shape)
integrated_df.head()

(120152, 20)


,MONTH_SQN,CUSTOMER_SQN,SUBSCRIPTION_SQN,SERVICE,CONTRACT_START_DATE,CONTRACT_END_DATE,CONTRACT_STATUS,SUBSCRIPTION_STATUS,NEW_SIGNUP_FLAG,RECONTRACT_FLAG,CHURN_FLAG,GENDER,AGE,RACE,NATIONALITY,CREDIT_RATING,num_cs_tickets,num_campaigns,num_promos,num_bundles
0,202404,11800042,23260741,Mobile,2023-07-25,2025-08-30,In Contract,Active,0,0,0,Male,43.0,Chinese,Singaporean,6.0,NaN,NaN,NaN,NaN
1,202406,11785940,15112297,MaxOnline - Consumer,2021-10-27,2023-12-02,Out Of Contract,Active,0,0,0,Female,43.0,Others,Singaporean,4.0,NaN,NaN,NaN,NaN
2,202405,11803623,26320787,Cable TV Residential,None,None,No Contract,Active,0,0,0,Female,44.0,Others,Singaporean,5.0,NaN,NaN,NaN,NaN
3,202404,11787339,22743979,Mobile,2023-03-26,2024-05-01,Out Of Contract,Active,0,0,0,Male,43.0,Chinese,Singaporean,6.0,1.0,NaN,NaN,NaN
4,202406,11793778,24847177,Mobile,None,None,No Contract,Inactive,0,0,1,Male,41.0,Chinese,Singaporean,7.0,5.0,NaN,NaN,NaN


In [6]:
# check if primary key is unique if assert fails post joins
assert (
    int(
        integrated_df.groupby(["MONTH_SQN", "CUSTOMER_SQN", "SUBSCRIPTION_SQN"])[
            "SERVICE"
        ]
        .count()
        .sort_values(ascending=False)
        .iloc[0]
    )
    == 1
), "Primary key is not unique"

## Data preparation for modeling using Pandas

In [7]:
# check for missing values
integrated_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120152 entries, 0 to 120151
Data columns (total 20 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   MONTH_SQN            120152 non-null  int64  
 1   CUSTOMER_SQN         120152 non-null  int64  
 2   SUBSCRIPTION_SQN     120152 non-null  int64  
 3   SERVICE              120152 non-null  object 
 4   CONTRACT_START_DATE  97572 non-null   object 
 5   CONTRACT_END_DATE    97572 non-null   object 
 6   CONTRACT_STATUS      120152 non-null  object 
 7   SUBSCRIPTION_STATUS  120152 non-null  object 
 8   NEW_SIGNUP_FLAG      120152 non-null  int64  
 9   RECONTRACT_FLAG      120152 non-null  int64  
 10  CHURN_FLAG           120152 non-null  int64  
 11  GENDER               79489 non-null   object 
 12  AGE                  79486 non-null   float64
 13  RACE                 79482 non-null   object 
 14  NATIONALITY          79489 non-null   object 
 15  CREDIT_RATING    

In [8]:
# rectify data type on date columns
integrated_df["CONTRACT_START_DATE"] = pd.to_datetime(
    integrated_df["CONTRACT_START_DATE"], format="%Y-%m-%d"
)
integrated_df["CONTRACT_END_DATE"] = pd.to_datetime(
    integrated_df["CONTRACT_END_DATE"], format="%Y-%m-%d"
)

# calculate contract duration
integrated_df["CONTRACT_DURATION"] = (
    (integrated_df["CONTRACT_END_DATE"] - integrated_df["CONTRACT_START_DATE"])
    .dt.days.fillna(-999)
    .astype(int)
)

# impute missing values
integrated_df = integrated_df.fillna(
    {
        "GENDER": "Unknown",
        "AGE": -999,
        "RACE": "Unknown",
        "NATIONALITY": "Unknown",
        "CREDIT_RATING": -999,
        "num_cs_tickets": 0,
        "num_campaigns": 0,
        "num_promos": 0,
        "num_bundles": 0,
    }
)

# update calculated columns to integer
integrated_df["num_cs_tickets"] = integrated_df["num_cs_tickets"].astype(int)
integrated_df["num_campaigns"] = integrated_df["num_campaigns"].astype(int)
integrated_df["num_promos"] = integrated_df["num_promos"].astype(int)
integrated_df["num_bundles"] = integrated_df["num_bundles"].astype(int)

print(integrated_df.shape)
integrated_df.head()

(120152, 21)


,MONTH_SQN,CUSTOMER_SQN,SUBSCRIPTION_SQN,SERVICE,CONTRACT_START_DATE,CONTRACT_END_DATE,CONTRACT_STATUS,SUBSCRIPTION_STATUS,NEW_SIGNUP_FLAG,RECONTRACT_FLAG,...,GENDER,AGE,RACE,NATIONALITY,CREDIT_RATING,num_cs_tickets,num_campaigns,num_promos,num_bundles,CONTRACT_DURATION
0,202404,11800042,23260741,Mobile,2023-07-25,2025-08-30,In Contract,Active,0,0,...,Male,43.0,Chinese,Singaporean,6.0,0,0,0,0,767
1,202406,11785940,15112297,MaxOnline - Consumer,2021-10-27,2023-12-02,Out Of Contract,Active,0,0,...,Female,43.0,Others,Singaporean,4.0,0,0,0,0,766
2,202405,11803623,26320787,Cable TV Residential,NaT,NaT,No Contract,Active,0,0,...,Female,44.0,Others,Singaporean,5.0,0,0,0,0,-999
3,202404,11787339,22743979,Mobile,2023-03-26,2024-05-01,Out Of Contract,Active,0,0,...,Male,43.0,Chinese,Singaporean,6.0,1,0,0,0,402
4,202406,11793778,24847177,Mobile,NaT,NaT,No Contract,Inactive,0,0,...,Male,41.0,Chinese,Singaporean,7.0,5,0,0,0,-999


In [9]:
# functions for feature engineering

# categorize contract duration
def contract_duration_banding(contract_duration):
    if contract_duration < 0:
        return "Not Applicable"
    elif contract_duration < 365:
        return "<1 year"
    elif contract_duration < 730:
        return "1-2 years"
    else:
        return ">2 years"


# categorize age
def age_banding(age):
    if age < 0:
        return "Unknown"
    elif 40 <= age < 51:
        return "40-50 Yrs"
    elif 51 <= age < 61:
        return "51-60 Yrs"
    elif 61 <= age < 71:
        return "61-70 Yrs"
    else:
        return "Above 70 Yrs"


# categorize credit rating
def credit_rating_banding(credit_rating):
    if credit_rating < 0:
        return "Unknown"
    elif credit_rating >= 4:
        return "High Value"
    elif credit_rating >= 2:
        return "Medium Value"
    else:
        return "Low Value"


# apply functions
integrated_df["CONTRACT_DURATION_BAND"] = integrated_df["CONTRACT_DURATION"].apply(
    contract_duration_banding
)
integrated_df["CREDIT_RATING_BAND"] = integrated_df["CREDIT_RATING"].apply(
    credit_rating_banding
)
integrated_df["AGE_BAND"] = integrated_df["AGE"].apply(age_banding)

# drop original columns & any nulls
integrated_df = integrated_df.drop(
    [
        "AGE",
        "CREDIT_RATING",
        "CONTRACT_DURATION",
        "CONTRACT_START_DATE",
        "CONTRACT_END_DATE",
    ],
    axis=1,
)

print(integrated_df.shape)
integrated_df.head()

(120152, 19)


,MONTH_SQN,CUSTOMER_SQN,SUBSCRIPTION_SQN,SERVICE,CONTRACT_STATUS,SUBSCRIPTION_STATUS,NEW_SIGNUP_FLAG,RECONTRACT_FLAG,CHURN_FLAG,GENDER,RACE,NATIONALITY,num_cs_tickets,num_campaigns,num_promos,num_bundles,CONTRACT_DURATION_BAND,CREDIT_RATING_BAND,AGE_BAND
0,202404,11800042,23260741,Mobile,In Contract,Active,0,0,0,Male,Chinese,Singaporean,0,0,0,0,>2 years,High Value,40-50 Yrs
1,202406,11785940,15112297,MaxOnline - Consumer,Out Of Contract,Active,0,0,0,Female,Others,Singaporean,0,0,0,0,>2 years,High Value,40-50 Yrs
2,202405,11803623,26320787,Cable TV Residential,No Contract,Active,0,0,0,Female,Others,Singaporean,0,0,0,0,Not Applicable,High Value,40-50 Yrs
3,202404,11787339,22743979,Mobile,Out Of Contract,Active,0,0,0,Male,Chinese,Singaporean,1,0,0,0,1-2 years,High Value,40-50 Yrs
4,202406,11793778,24847177,Mobile,No Contract,Inactive,0,0,1,Male,Chinese,Singaporean,5,0,0,0,Not Applicable,High Value,40-50 Yrs


In [10]:
# examining distribution of churn flag (target variable for prediction)
integrated_df["CHURN_FLAG"].value_counts(normalize=True)

CHURN_FLAG
0    0.980458
1    0.019542
Name: proportion, dtype: float64

In [11]:
# extract final data for analysis on Tableau
integrated_df.to_csv("../data/integrated_df.csv", index=False)

## Churn Prediction Modeling using Sci-Kit Learn

In [12]:
# separate features and target variable
X = integrated_df.drop(
    columns=[
        "CUSTOMER_SQN",
        "MONTH_SQN",
        "SUBSCRIPTION_SQN",
        "CONTRACT_STATUS",
        "SUBSCRIPTION_STATUS",
        "CHURN_FLAG",
    ]
)  # Features
y = integrated_df["CHURN_FLAG"]  # Target

In [13]:
# pre-processing pipeline to handle categorical and numerical features as required by model
numeric_features = ["num_cs_tickets", "num_campaigns", "num_promos", "num_bundles"]
numeric_transformer = StandardScaler()

categorical_features = [
    "SERVICE",
    "GENDER",
    "RACE",
    "NATIONALITY",
    "CONTRACT_DURATION_BAND",
    "CREDIT_RATING_BAND",
    "AGE_BAND",
]
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ],
    remainder="passthrough",
)

clf = make_pipeline(
    preprocessor, LogisticRegression(class_weight="balanced", random_state=42)
)

In [14]:
# model training
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

clf.fit(X_train, y_train)

/home/shilpa/miniforge3/envs/interview/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/shilpa/miniforge3/envs/interview/lib/python3.11/site-packages/sklearn/compose/_column_transformer.py:1623: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column na

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['num_cs_tickets',
                                                   'num_campaigns',
                                                   'num_promos',
                                                   'num_bundles']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['SERVICE', 'GENDER', 'RACE',
                                                   'NATIONALITY',
                                                   'CONTRACT_DURATION_BAND',
                                                   'CREDIT_RATING_BAND',
                                                   'AGE_BAND'])])),
                ('logisticregression',
                 LogisticRegression(class_weight='balanced', random_state=42))])

In [15]:
# model evaluation using recall
for metric in ["recall"]:
    scorer = get_scorer(metric)
    train_score = scorer(clf, X_train, y_train)
    test_score = scorer(clf, X_test, y_test)
    print(f"train_{metric}: {train_score:.2f}")
    print(f"test_{metric}: {test_score:.2f}")

train_recall: 0.68
test_recall: 0.62


In [16]:
# get the feature names
numeric_feature_names = numeric_features
categorical_feature_names = (
    clf.named_steps["columntransformer"]
    .named_transformers_["cat"]
    .get_feature_names_out(categorical_features)
)

# get passthrough columns
passthrough_columns = clf.named_steps["columntransformer"].transformers_[2][2]
passthrough_feature_names = X_train.columns[passthrough_columns]

# combine all feature names
all_feature_names = np.concatenate(
    [numeric_feature_names, categorical_feature_names, passthrough_feature_names]
)

/home/shilpa/miniforge3/envs/interview/lib/python3.11/site-packages/sklearn/compose/_column_transformer.py:1623: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


In [17]:
# extract the LogisticRegression model from the pipeline
logistic_regression_model = clf.named_steps["logisticregression"]

# get the coefficients from the logistic regression model
coefficients = logistic_regression_model.coef_[0]

In [18]:
# create a DataFrame for better visualization
feature_importance = pd.DataFrame(
    {"Feature": all_feature_names, "Coefficient": coefficients}
)

# sort by absolute value of coefficient (higher value means more important influence on churn)
feature_importance["AbsCoefficient"] = feature_importance["Coefficient"].abs()
feature_importance = feature_importance.sort_values(
    by="AbsCoefficient", ascending=False
)

feature_importance.head()

,Feature,Coefficient,AbsCoefficient
45,NATIONALITY_Netherlander,3.538015,3.538015
58,NATIONALITY_Taiwanese,-3.357841,3.357841
77,RECONTRACT_FLAG,-2.759024,2.759024
32,NATIONALITY_German,2.596297,2.596297
54,NATIONALITY_South Korean,-1.584143,1.584143


In [19]:
# export for analysis on Tableau
feature_importance.to_csv("../data/ml_feature_importance.csv", index=False)

## Tableau Dashboard Deployment

In [20]:
from tabpy.tabpy_tools.client import Client

client = Client("http://localhost:6311/")

def Churn_Prediction(
    _arg1,
    _arg2,
    _arg3,
    _arg4,
    _arg5,
    _arg6,
    _arg7,
    _arg8,
    _arg9,
    _arg10,
    _arg11,
    _arg12,
    _arg13,
):
    input_data = np.column_stack(
        [
            _arg1,
            _arg2,
            _arg3,
            _arg4,
            _arg5,
            _arg6,
            _arg7,
            _arg8,
            _arg9,
            _arg10,
            _arg11,
            _arg12,
            _arg13,
        ]
    )
    X = pd.DataFrame(
        input_data,
        columns=[
            "SERVICE",
            "NEW_SIGNUP_FLAG",
            "RECONTRACT_FLAG",
            "GENDER",
            "RACE",
            "NATIONALITY",
            "num_cs_tickets",
            "num_campaigns",
            "num_promos",
            "num_bundles",
            "CONTRACT_DURATION_BAND",
            "CREDIT_RATING_BAND",
            "AGE_BAND",
        ],
    )
    result = clf.predict_proba(X)
    return result[:, 1].tolist()[0]


client.deploy(
    "Customer Churn Prediction",
    Churn_Prediction,
    "Returns prediction probability for customer churning.",
    override=True,
)